<a href="https://colab.research.google.com/github/Fuenfgeld/Agent_Tutorial_PydanticAI/blob/main/03_SystemPromts_Tools_PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install pydantic-ai
%pip -q install nest_asyncio
%pip -q install logfire

In [ ]:
import os
from google.colab import userdata

keyAntropic = userdata.get('Claude')
keyOpenAI = userdata.get('openAI')
keyLogFire = userdata.get('logfire')


os.environ["OPENAI_API_KEY"] = keyOpenAI
os.environ["ANTHROPIC_API_KEY"] = keyAntropic

import nest_asyncio
nest_asyncio.apply()
#logfire.configure(token=keyLogFire)

# System Promt

Generally, system prompts fall into two categories

   * Static system prompts: These are known when writing the code and can be defined via the system_prompt parameter of the Agent constructor.
   * Dynamic system prompts: These depend in some way on context that isn't known until runtime, and should be defined via functions decorated with @agent.system_prompt.



In [ ]:
import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from datetime import date

# Define the model
model = OpenAIModel('gpt-4o-mini')

# Create an agent that will use GPT-4o as the LLM
# The deps_type=str means we'll pass a string as dependency (patient ID)
agent = Agent(
    model,
    deps_type=str,
    system_prompt="Use the patient's ID when discussing their medical information.",
)

# Add system prompt that includes the patient ID in the context
@agent.system_prompt
def add_patient_id(ctx: RunContext[str]) -> str:
    return f"The patient's ID you're discussing is {ctx.deps}."

# Add system prompt that includes today's date for medical record keeping
@agent.system_prompt
def add_current_date() -> str:
    return f'Today\'s date for the medical record is {date.today()}.'

# Run the agent with a query about lab results and include patient ID
result = agent.run_sync('What were my latest blood glucose results? also what date is it', deps='PT12345')
print(result.output)


# Tools